In [1]:
!mkdir -p /content/decathlon_data
!wget http://www.robots.ox.ac.uk/~vgg/share/decathlon-1.0-data.tar.gz -O /content/decathlon_data/decathlon-1.0-data.tar.gz


--2024-11-13 16:14:48--  http://www.robots.ox.ac.uk/~vgg/share/decathlon-1.0-data.tar.gz
Resolving www.robots.ox.ac.uk (www.robots.ox.ac.uk)... 129.67.94.2
Connecting to www.robots.ox.ac.uk (www.robots.ox.ac.uk)|129.67.94.2|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.robots.ox.ac.uk/~vgg/share/decathlon-1.0-data.tar.gz [following]
--2024-11-13 16:14:48--  https://www.robots.ox.ac.uk/~vgg/share/decathlon-1.0-data.tar.gz
Connecting to www.robots.ox.ac.uk (www.robots.ox.ac.uk)|129.67.94.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 406351554 (388M) [application/x-gzip]
Saving to: ‘/content/decathlon_data/decathlon-1.0-data.tar.gz’

/content/decathlon_ 100%[===================>] 387.53M  21.3MB/s    in 20s     

2024-11-13 16:15:09 (19.7 MB/s) - ‘/content/decathlon_data/decathlon-1.0-data.tar.gz’ saved [406351554/406351554]



In [2]:
import tarfile

with tarfile.open('/content/decathlon_data/decathlon-1.0-data.tar.gz') as tar:
    tar.extractall(path='/content/decathlon_data')


In [3]:

with tarfile.open('/content/decathlon_data/aircraft.tar') as tar:
    tar.extractall(path='/content/decathlon_data/')


In [4]:
# Cell 1: Import necessary libraries and set up device
import torch
from torch.optim.lr_scheduler import MultiStepLR
import torch.nn as nn
from torchvision import models, transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder

# Set up device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cuda


In [5]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

train_dir = '/content/decathlon_data/aircraft/train'
val_dir = '/content/decathlon_data/aircraft/val'

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train_dataset = datasets.ImageFolder(train_dir, transform=transform)
val_dataset = datasets.ImageFolder(val_dir, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)


In [6]:
import torch
import torch.nn as nn
import torchvision.models as models

# resnet50
model = models.resnet50(pretrained=True)

# final fc layer
num_classes = len(train_dataset.classes)
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to('cuda')


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 191MB/s]


In [18]:
import torch.optim as optim

from torch.optim.lr_scheduler import MultiStepLR

num_epochs = 20  # 150
initial_lr = 0.001
milestones = [5, 10, 15]
gamma = 0.1

optimizer = torch.optim.SGD(model.parameters(), lr=initial_lr, momentum=0.9)
scheduler = MultiStepLR(optimizer, milestones=milestones, gamma=gamma)
criterion = torch.nn.CrossEntropyLoss()


In [19]:
import torch.cuda.amp as amp
scaler = amp.GradScaler()  # mixed-precision scaler

# Cell 3: Training and validation functions
def train_one_epoch(epoch):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        with torch.cuda.amp.autocast():  # Mixed precision
            outputs = model(images)
            loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch} - Training loss: {running_loss / len(train_loader)}")

def validate():
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Validation Accuracy: {accuracy}%")
    return accuracy


<ipython-input-19-42204bd81e0f>:2: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = amp.GradScaler()  # mixed-precision scaler


In [20]:
# Cell 4: Training loop with learning rate drops
best_accuracy = 0
for epoch in range(num_epochs):
    train_one_epoch(epoch)
    accuracy = validate()
    scheduler.step()  # Adjust learning rate as per the schedule

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        torch.save(model.state_dict(), "best_finetuned_model.pth")  # Save best model

    print(f"Epoch {epoch+1}/{num_epochs} completed. Best Accuracy so far: {best_accuracy}%")

<ipython-input-19-42204bd81e0f>:12: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():  # Mixed precision


Epoch 0 - Training loss: 4.615468726967865
Validation Accuracy: 2.43024302430243%
Epoch 1/20 completed. Best Accuracy so far: 2.43024302430243%
Epoch 1 - Training loss: 4.456771562684257
Validation Accuracy: 5.6705670567056705%
Epoch 2/20 completed. Best Accuracy so far: 5.6705670567056705%
Epoch 2 - Training loss: 4.260826641658567
Validation Accuracy: 8.4008400840084%
Epoch 3/20 completed. Best Accuracy so far: 8.4008400840084%
Epoch 3 - Training loss: 4.010825202150165
Validation Accuracy: 12.211221122112212%
Epoch 4/20 completed. Best Accuracy so far: 12.211221122112212%
Epoch 4 - Training loss: 3.7188505991449894
Validation Accuracy: 14.641464146414641%
Epoch 5/20 completed. Best Accuracy so far: 14.641464146414641%
Epoch 5 - Training loss: 3.494002382710295
Validation Accuracy: 15.661566156615661%
Epoch 6/20 completed. Best Accuracy so far: 15.661566156615661%
Epoch 6 - Training loss: 3.4613864556798397
Validation Accuracy: 15.931593159315932%
Epoch 7/20 completed. Best Accuracy 

In [21]:
# new lr
learning_rates = [0.01, 0.1]
results = {}
num_epochs = 20


In [22]:

def reset_model():
    model = models.resnet50(pretrained=True)
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    return model.to(device)

for lr in learning_rates:
    print(f"\nStarting experiment with learning rate = {lr}")
    model = reset_model()

    optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    scheduler = MultiStepLR(optimizer, milestones=milestones, gamma=gamma)

    best_accuracy = 0

    for epoch in range(num_epochs):
        train_one_epoch(epoch)
        accuracy = validate()
        scheduler.step()

        if accuracy > best_accuracy:
            best_accuracy = accuracy
            torch.save(model.state_dict(), f"best_finetuned_model_lr_{lr}.pth")

        print(f"Epoch {epoch+1}/{num_epochs} completed. Best Accuracy so far with LR {lr}: {best_accuracy}%")

    results[lr] = best_accuracy

for lr, accuracy in results.items():
    print(f"Final best accuracy with learning rate {lr}: {accuracy}%")




Starting experiment with learning rate = 0.01


<ipython-input-19-42204bd81e0f>:12: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():  # Mixed precision


Epoch 0 - Training loss: 4.400332806245336
Validation Accuracy: 12.691269126912692%
Epoch 1/20 completed. Best Accuracy so far with LR 0.01: 12.691269126912692%
Epoch 1 - Training loss: 2.8784303620176495
Validation Accuracy: 25.442544254425442%
Epoch 2/20 completed. Best Accuracy so far with LR 0.01: 25.442544254425442%
Epoch 2 - Training loss: 1.6528653153833353
Validation Accuracy: 37.29372937293729%
Epoch 3/20 completed. Best Accuracy so far with LR 0.01: 37.29372937293729%
Epoch 3 - Training loss: 0.9895726667260224
Validation Accuracy: 36.27362736273627%
Epoch 4/20 completed. Best Accuracy so far with LR 0.01: 37.29372937293729%
Epoch 4 - Training loss: 0.5170760329039592
Validation Accuracy: 47.64476447644765%
Epoch 5/20 completed. Best Accuracy so far with LR 0.01: 47.64476447644765%
Epoch 5 - Training loss: 0.1896340951042355
Validation Accuracy: 56.58565856585658%
Epoch 6/20 completed. Best Accuracy so far with LR 0.01: 56.58565856585658%
Epoch 6 - Training loss: 0.1066656055

Three learning rates were compared: 0.001, 0.01, and 0.1. At 0.001, the model converged steadily, achieving a best accuracy of 18.06%, indicating slow but stable learning. With 0.01, accuracy improved significantly to 57.61%, showing that a higher rate accelerated convergence and improved performance, However, with 0.1, accuracy dropped to 7.17%, likely due to excessive updates that hindered convergence. Overall, 0.01 provided the best results, balancing faster learning with stable convergence.


In [10]:
import torch
import torch.nn as nn
import torchvision.models as models

# Load the pre-trained ResNet50 model
model = models.resnet50(pretrained=True)

# Freeze all layers except the final fully connected layer
for param in model.parameters():
    param.requires_grad = False

# Update the final fully connected layer to match the target dataset
num_classes = len(train_dataset.classes)  # Make sure train_dataset is loaded and defined
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


In [11]:
import torch.optim as optim
from torch.optim.lr_scheduler import MultiStepLR

# Training function for one epoch
def train_one_epoch(epoch, dataloader, criterion, optimizer):
    model.train()
    running_loss = 0.0
    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(dataloader.dataset)
    print(f"Epoch {epoch} - Training loss: {epoch_loss}")

# Validation function
def validate(dataloader, criterion):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    accuracy = 100 * correct / total
    print(f"Validation Accuracy: {accuracy}%")
    return accuracy


In [12]:
# Define initial hyperparameters
learning_rates = [0.01, 0.001] #[0.1,0.01,0.001]
num_epochs = 20
milestones = [5, 10, 15]

for lr in learning_rates:
    print(f"\nStarting experiment with learning rate = {lr}")
    optimizer = optim.SGD(model.fc.parameters(), lr=lr, momentum=0.9)
    scheduler = MultiStepLR(optimizer, milestones=milestones, gamma=0.1)
    criterion = nn.CrossEntropyLoss()

    best_accuracy = 0
    for epoch in range(num_epochs):
        train_one_epoch(epoch, train_loader, criterion, optimizer)  # Assume train_loader is defined
        accuracy = validate(val_loader, criterion)  # Assume val_loader is defined

        # Save the best model for each learning rate
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            torch.save(model.state_dict(), f"best_feature_extractor_lr_{lr}.pth")

        scheduler.step()
    print(f"Best Accuracy with LR={lr}: {best_accuracy}%")



Starting experiment with learning rate = 0.01
Epoch 0 - Training loss: 4.634102731245895
Validation Accuracy: 5.4005400540054005%
Epoch 1 - Training loss: 4.177594509536661
Validation Accuracy: 7.890789078907891%
Epoch 2 - Training loss: 3.755975847696214
Validation Accuracy: 11.761176117611761%
Epoch 3 - Training loss: 3.420887715386
Validation Accuracy: 13.111311131113112%
Epoch 4 - Training loss: 3.15342857298482
Validation Accuracy: 13.861386138613861%
Epoch 5 - Training loss: 2.837767365312033
Validation Accuracy: 18.391839183918393%
Epoch 6 - Training loss: 2.7517524832512135
Validation Accuracy: 18.391839183918393%
Epoch 7 - Training loss: 2.726473193720707
Validation Accuracy: 18.6018601860186%
Epoch 8 - Training loss: 2.7085370235122745
Validation Accuracy: 18.57185718571857%
Epoch 9 - Training loss: 2.6894915402829467
Validation Accuracy: 18.511851185118513%
Epoch 10 - Training loss: 2.65576637170239
Validation Accuracy: 18.661866186618663%
Epoch 11 - Training loss: 2.649722

In feature extraction with a frozen ResNet50 model, learning rates of 0.1, 0.01, and 0.001 yielded distinct results in validation accuracy and training dynamics. Learning rate 0.1 achieved the highest validation accuracy at 19.53%, though training loss fluctuated. Learning rate 0.01 reached a peak accuracy of 18.81% with more consistent loss. The lowest rate, 0.001, produced a peak accuracy of 19.14% with slow, steady reduction in loss over epochs, showing stable but gradual progress. Higher learning rates accelerated convergence due to updates in only the final layer.

full finetuning yielded a much higher final accuracy (~59%) compared to feature extraction (where the best accuracy reached around 19.53%) across all tested learning rates. The winning approach was full finetuning with a learning rate of 0.01, which showed strong performance, achieving the highest validation accuracy due to the model adjusting weights in all layers.

The main reason full finetuning outperformed feature extraction is that updating all layers allows the model to adapt more to the nuances of the new dataset, especially when the target dataset is distinct from the source dataset (ImageNet). By freezing most of the layers, feature extraction restricts adaptation, resulting in lower accuracy as it relies only on adjustments to the final layer. Full finetuning takes advantage of all layers’ ability to learn dataset-specific features.